In [7]:
#importing libraries 
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import mean_squared_error as mse

In [8]:
df = pd.read_csv("insurance.csv")

In [9]:
df.head(3)

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.90,0,yes,southwest,16884.9240
1,18,male,33.77,1,no,southeast,1725.5523
2,28,male,33.00,3,no,southeast,4449.4620


In [26]:
df.shape

(1338, 7)

In [25]:
#How to use random numbers to create train and test data if you don't want to use train test split


df['randomnum'] = np.random.randint(1,6,df.shape[0])

train_temp = df[df['randomnum'] >= 3]
test_temp = df[df.randomnum < 3] 

train_temp = train_temp.drop('randomnum', axis=1)
test_temp = test_temp.drop('randomnum', axis=1)

df = df.drop('randomnum', axis=1)

In [31]:
train_temp

,age,sex,bmi,children,smoker,region,charges
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
5,31,female,25.740,0,no,southeast,3756.62160
7,37,female,27.740,3,no,northwest,7281.50560
...,...,...,...,...,...,...,...
1332,52,female,44.700,3,no,southwest,11411.68500
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500
